![Titulo](Imagenes/Titulos/Head3-3.png)

## Formulación matricial

En el MEF, buscando aproximar una solución de la variable dependiente en nodos específicos, se producen un conjunto de ecuaciones simultaneas que deben de ser resueltas de forma directa o iterativa. Los ejemplos presentados hasta aquí han tenido un número relativamente pequeño de ecuaciones e incógnitas, por lo que ha sido relativamente fácil resolverlos a mano. Sin embargo, para la mayoría de los problemas, el número de nodos, y por lo tanto incógnitas, será mucho mayor por lo que será indispensable el uso de una computadora para encontrar la solución.

Los coeficientes de las matrices que hemos utilizado se obtienen de las evaluaciones locales (por elemento) y se ensamblan en arreglos globales (del dominio). En otras palabras, los coeficientes _"rellenan"_ una matriz grande que contiene las contribuciones de todos los elementos. Usando una computadora esto se puede hacer fácilmente usando _loops_ en el programa. Una vez que formulamos el algoritmo de elemento finito para un elemento la naturaleza del MEF nos permite usar el mismo procedimiento para todos los elementos. De esta manera podemos construir el conjunto de matrices globales a partir en la evaluación local (en cada uno de los elementos) y después resolver la ecuación matricial resultante usando el método que prefiramos.

Será conveniente definir las operaciones usando una notación matricial, que nos servirá en general para trabajar con las ecuaciones con un grado de abstracción mayor.

### Matriz de _masa_:

La matriz de _masa_ se obtiene integrando el producto cruz de las funciones de forma con sigo mismas y esta relacionada con la parte temporal de nuestra ecuación diferencial. Para un elemento $e_k$ la matriz $\mathbf{M}$ (de masa) queda:


$$\mathbf{M}^{(e_k)} = \begin{bmatrix} m_{ij}^{(e_k)} \end{bmatrix} = \int_0^{h^{(e_k)}} \begin{bmatrix} N_i^{(e_k)} & N_j^{(e_k)} \end{bmatrix} dx$$

Para todo el dominio quedaría:

$$\mathbf{M} = \begin{bmatrix} m_{ij} \end{bmatrix} = \sum_{k=1}^n \mathbf{M}^{(e_k)} = \sum_{k=1}^n \int_0^{h^{(e_k)}} \begin{bmatrix} N_i^{(e_k)} & N_j^{(e_k)} \end{bmatrix} dx$$

donde $n$ es el número de elementos. Para ser cierta esta sumatoria implica que los componentes $m_{ij}^{(e_k)}$ de los elementos serán colocados en los correspondientes lugares $m_{ij}$ de la matriz global. Esta deberá de tener (para el caso lineal) dimensiones $(n+1)\times(n+1)$.

Así, se puede pensar que las matrices $\mathbf{M}^{(e_k)}$ son __expandidas__ con ceros para tener las dimensiones de la matriz global antes de ser sumadas.

Como ejemplo, consideremos la discretización del intervalo $0 \le x \le 1$ usando tres elementos lineales de la misma longitud. La evaluación de las matrices de masa queda:

$$\mathbf{M}^{(e_1)} = \mathbf{M}^{(e_2)} = \mathbf{M}^{(e_3)} = \dfrac{1}{18} \begin{bmatrix} 2 & 1 \\ 1 & 2 \end{bmatrix}$$

La matriz global es:

$$\mathbf{M} = \begin{bmatrix} 1/9 & 1/18 & 0 & 0 \\ 1/18 & 1/9 & 0 & 0 \\  0&0&0&0 \\ 0&0&0&0 \end{bmatrix} + \begin{bmatrix} 0&0&0&0 \\ 0 & 1/9 & 1/18 &  0 \\ 0 & 1/18 & 1/9 &  0 \\ 0&0&0&0 \end{bmatrix} + \begin{bmatrix} 0&0&0&0 \\ 0&0&0&0 \\ 0 & 0 & 1/9 & 1/18 \\ 0&0 & 1/18 & 1/9 \end{bmatrix}$$

$$\mathbf{M} = \begin{bmatrix} 1/9 & 1/18 & 0 & 0 \\ 1/18 & 2/9 & 1/18 & 0 \\ 0& 1/18 & 2/9 & 1/18 \\ 0&0& 1/18 & 1/9 \end{bmatrix}$$

Este tipo de matrices son conocidas matrices dispersas (_sparse matrix_) por tener una buena cantidad de ceros. Normalmente, si hay una relación entre la numeración de los nodos y los elementos, se obtienen matrices en los cuales los ceros aparecen al alejarnos de la diagonal principal. Esto facilitará la solución numérica de estos sistemas.

## Matriz de _difusión_:

De manera similar a la anterior, del termino de difusión se obtiene la matriz de difusión $\mathbf{K}$:

$$\mathbf{K} = \sum_{k=1}^n \mathbf{K}^{(e_k)} = \sum_{k=1}^n \int_0^{h^{(e_k)}} K^{(e_k)} \begin{bmatrix} \dfrac{dN_i^{(e_k)}}{dx} & \dfrac{dN_j^{(e_k)}}{dx} \end{bmatrix} dx$$

Esta matriz es conocida también como matriz de rigidez (_stiffness_) o conductividad (_conductance_).

## Integral para valores conocidos:

La integral que contiene las contribuciones de funciones conocidas (como en el caso de la fuente de calor $Q$) se formulan como un vector, con el mismo número de nodos.

$$\mathbf{F} = \sum_{k=1}^n \mathbf{F}^{(e_k)} = \sum_{k=1}^n \int_0^{h^{(e_k)}} \begin{bmatrix} N_i^{(e_k)} \end{bmatrix} Q^{(e_k)} dx$$

Esta vector $\mathbf{F}$ se usa para fuerzas distribuidas, generación de temperatura, entre otros y se conoce normalmente como vector de carga (_load_).

En general las condiciones de frontera se añaden a la matriz de difusión $\mathbf{K}$ aparece la variable dependiente, o a el vector $\mathbf{F}$.

Usando la notación anterior, un sistema dependiente del tiempo se puede expresar como:

$$\mathbf{M}\dot{\mathbf{T}}+\mathbf{K}\mathbf{T} = \mathbf{F}$$

Donde:

* $\mathbf{T}$ es el vector de incógnitas.
* $\dot{\mathbf{T}}$ es el vector que contiene las derivadas con respecto al tiempo de $\mathbf{T}$

Siguiendo lo que vimos en el tema de sistemas no estacionarios, la solución de este problema se puede plantear como:

$$\left( \mathbf{M} + \theta \Delta t \mathbf{K} \right)\mathbf{T}^{n+1} = \left( \mathbf{M} + (1-\theta) \Delta t \mathbf{K} \right)\mathbf{T}^n + \Delta t \left(\theta \mathbf{F}^{n+1} + (1-\theta) \mathbf{F}^n\right)$$

Esto nos permite trabajar con estas expresiones usando el álgebra de matrices.

Por ejemplo, la solución $\mathbf{T}^{n+1}$ se puede obtener usando:

$$\mathbf{T}^{n+1} = \left( \mathbf{M} + \theta \Delta t \mathbf{K} \right)^{-1}\left[\left( \mathbf{M} + (1-\theta) \Delta t \mathbf{K} \right)\mathbf{T}^n + \Delta t \left(\theta \mathbf{F}^{n+1} + (1-\theta) \mathbf{F}^n\right)\right]$$

Siempre y cuando $\left( \mathbf{M} + \theta \Delta t \mathbf{K} \right)$ sea invertible y no singular.